In [1]:
from dotenv import load_dotenv
load_dotenv('/etc/proxy.env')

True

In [2]:
# !pip install transformers -U

In [3]:
# !pip install tf-keras

In [4]:
# !pip install 'accelerate>=0.26.0'

In [5]:
import torch
import pandas as pd
import numpy as np
import random
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report, confusion_matrix
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification
# from sklearn.model_selection import train_test_split

print(torch.cuda.is_available())
torch.__version__

2024-10-12 14:13:58.605376: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-12 14:13:58.627256: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-12 14:13:58.651386: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-12 14:13:58.658260: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-12 14:13:58.676873: I tensorflow/core/platform/cpu_feature_guar

True


'2.3.0'

In [6]:
num_labels =120
num_labels 

120

In [7]:
ids_to_classes = ['acadapter | ACAdapter Others',
       'acadapter | Adapter Cannot Charge System', 'acadapter | _others_',
       'audio | Audio Others', 'audio | Distorted Audio',
       'audio | Earphone no sound', 'audio | No Audio',
       'audio | Speaker no sound', 'audio | _others_',
       'battery | Battery Not Charging', 'battery | Battery Others',
       'battery | Short Life/ Discharge Quickly', 'battery | _others_',
       'bios | BIOS Others', 'bios | _others_', 'bluetooth | _others_',
       'bootup | No Boot', 'bootup | No Post', 'bootup | No Power',
       'bootup | _others_', 'bsod | BSOD Others',
       'bsod | BSOD-No details', 'bsod | _others_',
       'call text insufficient | Call text insufficient',
       "charging | System can't charging", 'charging | _others_',
       'dock | Dock station failed', 'dock | _others_',
       'hdd/ssd | Cannot install 2nd HDD/SSD',
       'hdd/ssd | HDD/SSD Cannot Detected', 'hdd/ssd | HDD/SSD Others',
       'hdd/ssd | HDD/SSD Unbootable', 'hdd/ssd | _others_',
       'keyboard | KB Backlight malfunction', 'keyboard | KB Not work',
       'keyboard | Key damaged', 'keyboard | Key fall',
       'keyboard | Key not work', 'keyboard | Keyboard Others',
       'keyboard | _others_', 'keyboard | key stuck',
       'lan | Lan port loose/abnormal', 'lan | _others_',
       'lcdpanelquality | Cracked Glass',
       'lcdpanelquality | LCDPanelQuality Others',
       'lcdpanelquality | Lines&Stripes', 'lcdpanelquality | Spots',
       'lcdpanelquality | _others_', 'mechanical | BackCover',
       'mechanical | Buttons malfunction', 'mechanical | Hinge',
       'mechanical | Mechanical Others', 'mechanical | _others_',
       'memory | Memory failed', 'memory | _others_',
       'microphone | Mic function abnormal', 'microphone | _others_',
       'mwd | Damage', 'mwd | _others_', 'noise | Fan noise(thermal)',
       'noise | Noise Others', 'noise | _others_',
       'peripherals keyboard | KB Not work',
       'peripherals keyboard | Key fall',
       'peripherals keyboard | Key not work',
       'peripherals keyboard | Keyboard Others',
       'peripherals keyboard | TP Click not working',
       'peripherals keyboard | TP Erratic Pointer',
       'peripherals keyboard | TP not work',
       'peripherals keyboard | TouchPad Others',
       'peripherals keyboard | _others_',
       'peripherals keyboard | key stuck', 'portslot | Audio port',
       'portslot | DC-IN', 'portslot | HDMI', 'portslot | RJ45',
       'portslot | Type-C', 'portslot | USB', 'portslot | _others_',
       'security | _others_', 'software | OS',
       'software | Software Others', 'software | _others_',
       'stylus | Stylus Others', 'stylus | _others_',
       'systemstability | Abnormal Reboot', 'systemstability | Freezing',
       'systemstability | Shut Down unexpected',
       'systemstability | Slow Performance',
       'systemstability | System Stability Others',
       'systemstability | System freezing', 'systemstability | _others_',
       'thermal | CPU high temp/skin temp', 'thermal | Fan not work',
       'thermal | Thermal Others', 'thermal | _others_',
       'touchpad | TP Click not working', 'touchpad | TP Erratic Pointer',
       'touchpad | TP not work', 'touchpad | TP stuck',
       'touchpad | TouchPad Others', 'touchpad | _others_',
       'touchpanel | Touch Panel No Response', 'touchpanel | _others_',
       'video | Abnormal color', 'video | Dim Video',
       'video | Distortion', 'video | Flickers', 'video | No video',
       'video | Video Others', 'video | _others_',
       'webcam | Camera No function', 'webcam | Webcam Others',
       'webcam | _others_', 'wireless | WLAN Cannot Connect',
       'wireless | WLAN Cannot Detect', 'wireless | Wireless Others',
       'wireless | _others_', 'wwan | SIM not Detected',
       'wwan | _others_']

In [8]:
len(ids_to_classes)

120

In [9]:
classes_to_ids = {class_name:i for i, class_name in enumerate(ids_to_classes)}
classes_to_ids

{'acadapter | ACAdapter Others': 0,
 'acadapter | Adapter Cannot Charge System': 1,
 'acadapter | _others_': 2,
 'audio | Audio Others': 3,
 'audio | Distorted Audio': 4,
 'audio | Earphone no sound': 5,
 'audio | No Audio': 6,
 'audio | Speaker no sound': 7,
 'audio | _others_': 8,
 'battery | Battery Not Charging': 9,
 'battery | Battery Others': 10,
 'battery | Short Life/ Discharge Quickly': 11,
 'battery | _others_': 12,
 'bios | BIOS Others': 13,
 'bios | _others_': 14,
 'bluetooth | _others_': 15,
 'bootup | No Boot': 16,
 'bootup | No Post': 17,
 'bootup | No Power': 18,
 'bootup | _others_': 19,
 'bsod | BSOD Others': 20,
 'bsod | BSOD-No details': 21,
 'bsod | _others_': 22,
 'call text insufficient | Call text insufficient': 23,
 "charging | System can't charging": 24,
 'charging | _others_': 25,
 'dock | Dock station failed': 26,
 'dock | _others_': 27,
 'hdd/ssd | Cannot install 2nd HDD/SSD': 28,
 'hdd/ssd | HDD/SSD Cannot Detected': 29,
 'hdd/ssd | HDD/SSD Others': 30,
 '

In [10]:
classes_to_ids['hdd/ssd | _others_']

32

In [11]:
model = BertForSequenceClassification.from_pretrained("output_1010_pure_english/2nd_train/checkpoint-23500")

In [12]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 512, padding_idx=0)
      (position_embeddings): Embedding(512, 512)
      (token_type_embeddings): Embedding(2, 512)
      (LayerNorm): LayerNorm((512,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-7): 8 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=512, out_features=512, bias=True)
              (key): Linear(in_features=512, out_features=512, bias=True)
              (value): Linear(in_features=512, out_features=512, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=512, out_features=512, bias=True)
              (LayerNorm): LayerNorm((512,), eps=1e-1

In [13]:
model = model.to('cuda')
model.device

device(type='cuda', index=0)

In [14]:
# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [15]:
# train_dataset_model = torch.load('output_1010_pure_english/train_dataset_model.pkt')
val_dataset_model = torch.load('output_1010_pure_english/val_dataset_model.pkt')

In [16]:
def compute_metrics(p):
    print(type(p))
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred, average='micro')
    precision = precision_score(y_true=labels, y_pred=pred, average='micro')
    f1 = f1_score(y_true=labels, y_pred=pred, average='micro')

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

def compute_metrics2(p):
    print(type(p))
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    report = classification_report(y_true=labels, y_pred=pred, target_names= ids_to_classes)
    print("Classification Report:\n", report)

    # cm = confusion_matrix(y_true=labels, y_pred=pred)
    # print("Confusion Matrix:\n", cm)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred, average='macro')
    precision = precision_score(y_true=labels, y_pred=pred, average='macro')
    f1 = f1_score(y_true=labels, y_pred=pred, average='macro')


In [17]:
# Define Trainer
args = TrainingArguments(
    output_dir="output_1010_pure_english/2nd_train",  # train 2nd after it was interupted. Previous output path is "output_1010_pure_english"
    num_train_epochs=5,
    per_device_train_batch_size=64

)

In [18]:
# trainer = Trainer(
#     model=model,
#     args=args,
#     train_dataset=train_dataset_model,
#     eval_dataset=val_dataset_model,
#     compute_metrics=compute_metrics
# )

In [19]:
# trainer with compute_metrics2
trainer2 = Trainer(
    model=model,
    args=args,
    # train_dataset=train_dataset_model,  # no need to speficy dataset here as we can specify data with evaluate(eval_dataset=your_evalDataset)
    # eval_dataset=val_dataset_model,
    compute_metrics=compute_metrics2
)

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [20]:
for param in model.parameters():
    param.data = param.data.contiguous()

In [21]:
# trainer.train()

In [22]:
# trainer.evaluate() # evaluate our model on the validation set

In [23]:
# val dataset can be specified, which overwrites self.eval_dataset
trainer2.evaluate(val_dataset_model)  

<class 'transformers.trainer_utils.EvalPrediction'>


ValueError: Number of classes, 114, does not match size of target_names, 120. Try specifying the labels parameter

In [ ]:
# ValueError: Number of classes, 114, does not match size of target_names, 120. Try specifying the labels parameter
